# Fashion MNIST via TensorFlow #

## Preliminaries ##

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization

import numpy as np
import matplotlib.pyplot as plt
import _pickle
import gzip
import os

with gzip.open('./data/fashion-mnist.pkl.gz', 'rb') as fp:
    (train_images, train_labels, test_images, test_labels) = \
    _pickle.load(fp)
print('Tensorflow version ' + tf.__version__)

Tensorflow version 2.1.0


In [2]:
%load_ext tensorboard

In [3]:
train_images_reshape = train_images.reshape(train_images.shape[0],
                                            28, 28, 1)
test_images_reshape = test_images.reshape(test_images.shape[0],
                                         28, 28, 1)

## Simplistic Network ##

In [4]:
model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [5]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(train_images, train_labels,
          validation_data=(test_images, test_labels),
          epochs=100,
          callbacks=[TensorBoard(
              log_dir=os.path.join('fitlog', 'simple'),
              histogram_freq=1,
              profile_batch = 100000000)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 10s 174us/sample - loss: 0.2311 - accuracy: 0.9140 - val_loss: 0.3410 - val_accuracy: 0.8809
Epoch 2/100
60000/60000 [==============================] - 9s 152us/sample - loss: 0.2254 - accuracy: 0.9162 - val_loss: 0.3270 - val_accuracy: 0.8884
Epoch 3/100
60000/60000 [==============================] - 9s 157us/sample - loss: 0.2165 - accuracy: 0.9190 - val_loss: 0.3421 - val_accuracy: 0.8838
Epoch 4/100
60000/60000 [==============================] - 7s 115us/sample - loss: 0.2112 - accuracy: 0.9208 - val_loss: 0.3510 - val_accuracy: 0.8758
Epoch 5/100
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2055 - accuracy: 0.9221 - val_loss: 0.3386 - val_accuracy: 0.8913
Epoch 6/100
60000/60000 [==============================] - 5s 90us/sample - loss: 0.1989 - accuracy: 0.9263 - val_loss: 0.3427 - val_accuracy: 0.8863
Epoch 7/100
60000/60000 [====================

Epoch 55/100
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0813 - accuracy: 0.9700 - val_loss: 0.5775 - val_accuracy: 0.8851
Epoch 56/100
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0786 - accuracy: 0.9709 - val_loss: 0.6082 - val_accuracy: 0.8806
Epoch 57/100
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0778 - accuracy: 0.9706 - val_loss: 0.5760 - val_accuracy: 0.8871
Epoch 58/100
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0783 - accuracy: 0.9710 - val_loss: 0.6150 - val_accuracy: 0.8864
Epoch 59/100
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0750 - accuracy: 0.9718 - val_loss: 0.6382 - val_accuracy: 0.8863
Epoch 60/100
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0753 - accuracy: 0.9718 - val_loss: 0.6226 - val_accuracy: 0.8860
Epoch 61/100
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0744 - ac

In [10]:
%tensorboard --logdir fitlog

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10700.

## Model 2 ##

In [11]:
model2 = Sequential([
    Conv2D(32, [3, 3], input_shape=(28, 28, 1), activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(32, [3, 3], activation='relu'),
    BatchNormalization(axis=-1),
    AveragePooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(64, [3, 3], activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(64, [3, 3], activation='relu'),
    BatchNormalization(axis=-1),
    AveragePooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [12]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
average_pooling2d (AveragePo (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)       

In [14]:
model2.fit(train_images_reshape,
          train_labels,
          validation_data=(test_images_reshape, test_labels),
          epochs=100,
          batch_size=25,
          callbacks=[TensorBoard(
              log_dir=os.path.join('fitlog','thin'),
              histogram_freq=1,
              profile_batch=100000000)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 285s 5ms/sample - loss: 0.5293 - accuracy: 0.8159 - val_loss: 0.3812 - val_accuracy: 0.8646
Epoch 2/100
60000/60000 [==============================] - 272s 5ms/sample - loss: 0.3488 - accuracy: 0.8741 - val_loss: 0.2859 - val_accuracy: 0.8959
Epoch 3/100
60000/60000 [==============================] - 272s 5ms/sample - loss: 0.3074 - accuracy: 0.8917 - val_loss: 0.2873 - val_accuracy: 0.8928
Epoch 4/100
60000/60000 [==============================] - 241s 4ms/sample - loss: 0.2779 - accuracy: 0.9010 - val_loss: 0.2543 - val_accuracy: 0.9093
Epoch 5/100
60000/60000 [==============================] - 266s 4ms/sample - loss: 0.2578 - accuracy: 0.9070 - val_loss: 0.2425 - val_accuracy: 0.9080
Epoch 6/100
60000/60000 [==============================] - 193s 3ms/sample - loss: 0.2423 - accuracy: 0.9132 - val_loss: 0.2341 - val_accuracy: 0.9144
Epoch 7/100
60000/60000 [===================

60000/60000 [==============================] - 178s 3ms/sample - loss: 0.0881 - accuracy: 0.9677 - val_loss: 0.2194 - val_accuracy: 0.9421
Epoch 55/100
60000/60000 [==============================] - 187s 3ms/sample - loss: 0.0870 - accuracy: 0.9675 - val_loss: 0.2142 - val_accuracy: 0.9392
Epoch 56/100
60000/60000 [==============================] - 183s 3ms/sample - loss: 0.0810 - accuracy: 0.9701 - val_loss: 0.2325 - val_accuracy: 0.9341
Epoch 57/100
60000/60000 [==============================] - 181s 3ms/sample - loss: 0.0845 - accuracy: 0.9692 - val_loss: 0.2197 - val_accuracy: 0.9366
Epoch 58/100
60000/60000 [==============================] - 180s 3ms/sample - loss: 0.0838 - accuracy: 0.9688 - val_loss: 0.2303 - val_accuracy: 0.9355
Epoch 59/100
60000/60000 [==============================] - 180s 3ms/sample - loss: 0.0826 - accuracy: 0.9696 - val_loss: 0.2299 - val_accuracy: 0.9377
Epoch 60/100
60000/60000 [==============================] - 180s 3ms/sample - loss: 0.0825 - accuracy

Personal best of
* loss: 0.1303
* acc: 0.9526
* val_loss: 0.2367
* val_acc: 0.9252

with a NN over 25 epochs:
* 2d CNN 32 (3, 3) relu
* normalization, axis=-1
* 2d CNN 32 (3, 3) relu
* normalization, axis=-1
* 2d max pooling (2, 2)
* dropout p = 0.25
* 2d CNN 64 (3, 3) relu
* normalization, axis=-1
* 2d CNN 64 (3, 3) relu
* normalization, axis=-1
* 2d max pooling (2, 2)
* dropout p = 0.25
* flatten
* dense 512 relu
* normalization
* dropout p = 0.5
* dense 10 softmax

This model is clearly overfitting, going to try some regularization

## Model 3 ##

In [15]:
model3 = Sequential([
    BatchNormalization(axis=-1, input_shape=(28, 28, 1)),
    Conv2D(64, [3, 3],
           activation='relu',
          bias_initializer='RandomNormal',
          kernel_initializer='random_uniform'),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D(512, [3, 3],
           activation='relu'),
    BatchNormalization(axis=-1),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128,
          activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(10, activation='softmax')
])

In [16]:
model3.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
              metrics=['accuracy'])

In [17]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_5 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
average_pooling2d_2 (Average (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 512)       295424    
_________________________________________________________________
batch_normalization_6 (Batch (None, 11, 11, 512)       2048      
_________________________________________________________________
average_pooling2d_3 (Average (None, 5, 5, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)            

In [18]:
model3.fit(train_images_reshape, train_labels,
          validation_data=(test_images_reshape, test_labels),
          epochs = 100,
          batch_size = 25,
          callbacks = [TensorBoard(
              log_dir = os.path.join('fitlog','wide'),
              histogram_freq = 1,
              profile_batch = 100000000)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 415s 7ms/sample - loss: 0.5886 - accuracy: 0.7904 - val_loss: 0.3629 - val_accuracy: 0.8677
Epoch 2/100
60000/60000 [==============================] - 480s 8ms/sample - loss: 0.3897 - accuracy: 0.8640 - val_loss: 0.3405 - val_accuracy: 0.8804
Epoch 3/100
60000/60000 [==============================] - 440s 7ms/sample - loss: 0.3395 - accuracy: 0.8815 - val_loss: 0.3050 - val_accuracy: 0.8901
Epoch 4/100
60000/60000 [==============================] - 432s 7ms/sample - loss: 0.3072 - accuracy: 0.8907 - val_loss: 0.2987 - val_accuracy: 0.8939
Epoch 5/100
60000/60000 [==============================] - 425s 7ms/sample - loss: 0.2874 - accuracy: 0.8967 - val_loss: 0.2981 - val_accuracy: 0.8891
Epoch 6/100
60000/60000 [==============================] - 423s 7ms/sample - loss: 0.2689 - accuracy: 0.9049 - val_loss: 0.2779 - val_accuracy: 0.9021
Epoch 7/100
60000/60000 [===================

60000/60000 [==============================] - 339s 6ms/sample - loss: 0.0852 - accuracy: 0.9718 - val_loss: 0.4737 - val_accuracy: 0.9221
Epoch 55/100
60000/60000 [==============================] - 290s 5ms/sample - loss: 0.0808 - accuracy: 0.9736 - val_loss: 0.5255 - val_accuracy: 0.9189
Epoch 56/100
60000/60000 [==============================] - 307s 5ms/sample - loss: 0.0786 - accuracy: 0.9744 - val_loss: 0.5589 - val_accuracy: 0.9198
Epoch 57/100
60000/60000 [==============================] - 378s 6ms/sample - loss: 0.0781 - accuracy: 0.9737 - val_loss: 0.4818 - val_accuracy: 0.9092
Epoch 58/100
60000/60000 [==============================] - 483s 8ms/sample - loss: 0.0761 - accuracy: 0.9749 - val_loss: 0.5724 - val_accuracy: 0.9121
Epoch 59/100
60000/60000 [==============================] - 415s 7ms/sample - loss: 0.0804 - accuracy: 0.9739 - val_loss: 0.5430 - val_accuracy: 0.9165
Epoch 60/100
60000/60000 [==============================] - 424s 7ms/sample - loss: 0.0781 - accuracy